# Use of PSF simulations code

In [ ]:
from simu_PSF_polar import *
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
device = torch.device('cuda')

In [ ]:
N_photons=1000 # number of photons collected 
N=80 # discretization of the BFP
l_pixel=4 #pixel size in micrometer
NA=1.4 # numerical aperture
mag=100 # first magnification
lambd=617 # wavelength
f_tube=200 # tube lens focal
MAG=200/150 # second magnification

In [ ]:
N_photons=torch.tensor(N_photons, device=device)
N=torch.tensor(N, device=device)
l_pixel=torch.tensor(l_pixel, device=device)
NA=torch.tensor(NA, device=device)
mag=torch.tensor(mag, device=device)
lambd=torch.tensor(lambd, device=device)
f_tube=torch.tensor(f_tube, device=device)
MAG=torch.tensor(MAG, device=device)

In [ ]:
NPSF = 80

In [ ]:
xp = torch.tensor([0. for i in range(NPSF)], device=device) # dipole position
yp = torch.tensor([0. for i in range(NPSF)], device=device) # dipole position
d_ = torch.tensor(-np.linspace(.1, 5, NPSF), device=device) # z position of dipole  
z = torch.tensor(np.array([1. for i in range(NPSF)]), device=device) # defocus of dipole in lambda units 
rho = torch.tensor([45. for i in range(NPSF)], device=device) # equatorial angle in degree
eta = torch.tensor([45. for i in range(NPSF)], device=device)  # azimuthal angle in degree
delta = torch.tensor([60. for i in range(NPSF)], device=device)  # wobbling in degree

In [ ]:
x, y, th1, phi, [Ex0, Ex1, Ex2], [Ey0, Ey1, Ey2], r, r_cut, k, f_o = vectorial_BFP_perfect_focus(N, NA=NA, mag=mag, lambd=lambd, f_tube=f_tube, device=device)

In [ ]:
u, v, M = compute_M(xp=xp, yp=yp, zp=z, d=d_, x=x, y=y, th1=th1, phi=phi, Ex0=Ex0, Ex1=Ex1, Ex2=Ex2
                    , Ey0=Ey0, Ey1=Ey1, Ey2=Ey2, r=r, r_cut=r_cut, k=k, f_o=f_o, N=N, l_pixel=l_pixel, NA=NA, mag=mag, lambd=lambd, f_tube=f_tube, MAG=MAG, device=device)

In [ ]:
M.shape

In [ ]:
psf = PSF(rho=rho, eta=eta, delta=delta, M=M, N_photons=N_photons)

In [ ]:
psf.shape

In [ ]:
# the result is of dimension 3 (polar, x_image, y_image) or 4 if rho ... are tensors of dimension N (several psf computed)
# then the dimension is () (N, polar, x_image, y_image)

In [ ]:
psfx = np.max(psf[:,0].cpu().detach().numpy(), axis=(0,))
psfy = np.max(psf[:,1].cpu().detach().numpy(), axis=(0,))
psfx = psf[25,0].cpu().detach().numpy()
psfy = psf[25,1].cpu().detach().numpy()

psf_unpolarized = psfx+psfy
posx = xp[0].cpu().detach().numpy()*mag.cpu().detach().numpy()*MAG.cpu().detach().numpy()
posy = yp[0].cpu().detach().numpy()*mag.cpu().detach().numpy()*MAG.cpu().detach().numpy()
posz = z[0].cpu().detach().numpy()
maag = (mag*MAG).cpu().detach().numpy()

In [ ]:
psf_radius = 0.001*maag* 0.61 * lambd.cpu().detach().numpy() / (NA.cpu().detach().numpy())  # in micrometer
x_circle = psf_radius * np.cos(np.linspace(0, 2 * np.pi, 100))
y_circle = psf_radius * np.sin(np.linspace(0, 2 * np.pi, 100))

In [ ]:
x_lim = 100
plt.rcParams['figure.figsize'] = [23, 5]
fig, ax = plt.subplots(1,3)


mesh = ax[0].pcolormesh(u.cpu().detach().numpy(), v.cpu().detach().numpy(), psfx, cmap='gray')
#ax[0].plot(x_circle, y_circle, label=' Theoretical PSF size', color='red')
#ax[0].legend()
ax[0].set_xlim((posx-x_lim, posx+x_lim))
ax[0].set_ylim((posy-x_lim, posy+x_lim))
ax[0].set_xlabel('x_camera ($\\mu$m)')
ax[0].set_ylabel('y_camera ($\\mu$m)')
ax[0].set_title('x polarization')
secax = ax[0].secondary_xaxis('top', functions=(lambda x : 1000*x/maag, lambda x : 0.001*maag*x))
secax2 = ax[0].secondary_yaxis('right', functions=(lambda x : 1000*x/maag, lambda x : 0.001*maag*x))
secax.set_xlabel('x_sample (nm)')
secax2.set_ylabel('y_sample (nm)')
cb = plt.colorbar(mesh, pad=0.15, label='Photon number')

mesh1 = ax[1].pcolormesh(u.cpu().detach().numpy(), v.cpu().detach().numpy(), psfy, cmap='gray')
#ax[1].plot(x_circle, y_circle, label=' Theoretical PSF size', color='red')
#ax[1].legend()
ax[1].set_xlim((posx-x_lim, posx+x_lim))
ax[1].set_ylim((posy-x_lim, posy+x_lim))
ax[1].set_xlabel('x_camera ($\\mu$m)')
ax[1].set_ylabel('y_camera ($\\mu$m)')
ax[1].set_title('y polarization')
secax = ax[1].secondary_xaxis('top', functions=(lambda x : 1000*x/maag, lambda x : 0.001*maag*x))
secax2 = ax[1].secondary_yaxis('right', functions=(lambda x : 1000*x/maag, lambda x : 0.001*maag*x))
secax.set_xlabel('x_sample (nm)')
secax2.set_ylabel('y_sample (nm)')
cb = plt.colorbar(mesh1, pad=0.15, label='Photon number')

mesh2 = ax[2].pcolormesh(u.cpu().detach().numpy(), v.cpu().detach().numpy(), psf_unpolarized, cmap='gray')
#ax[2].plot(x_circle, y_circle, label=' Theoretical PSF size', color='red')
#ax[2].legend()
ax[2].set_xlim((posx-x_lim, posx+x_lim))
ax[2].set_ylim((posy-x_lim, posy+x_lim))
ax[2].set_xlabel('x_camera ($\\mu$m)')
ax[2].set_ylabel('y_camera ($\\mu$m)')
ax[2].set_title('unpolarized')
secax = ax[2].secondary_xaxis('top', functions=(lambda x : 1000*x/maag, lambda x : 0.001*maag*x))
secax2 = ax[2].secondary_yaxis('right', functions=(lambda x : 1000*x/maag, lambda x : 0.001*maag*x))
secax.set_xlabel('x_sample (nm)')
secax2.set_ylabel('y_sample (nm)')
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')

In [ ]:
uu, zz = np.meshgrid(u[0].cpu(), -d_.cpu().numpy())

psfx = psf[:,0,int(psf.shape[3]/2)].cpu().detach().numpy()
psfy = psf[:,1,int(psf.shape[3]/2)].cpu().detach().numpy()
psf_unpolarized = psfx+psfy

In [ ]:
x_lim = 2000#800
z_lim = 2000
plt.rcParams['figure.figsize'] = [19, 5]
fig, ax = plt.subplots(1,3)


mesh1 = ax[0].pcolormesh(1000*uu/maag, 1000*zz, psfx, cmap='gray', norm=LogNorm(vmin=1e-2, vmax=1e2))
ax[0].axhline(-1000*d_.cpu().numpy()[25], alpha=0.7)
ax[0].scatter([0.], 1000*z.cpu().detach().numpy()[0], label='Emitter position', c='r', marker='+')
ax[0].legend()
ax[0].set_xlim((posx-x_lim, posx+x_lim))
#ax[0].set_ylim((posz*1000-z_lim, posz*1000+z_lim))
ax[0].set_aspect('equal')
ax[0].set_xlabel('x_sample (nm)')
ax[0].set_ylabel('z (nm)')
ax[0].set_title('x polarization')
secax = ax[0].secondary_xaxis('top', functions=(lambda x : 0.001*maag*x, lambda x : 1000*x/maag))
secax.set_xlabel('x_camera ($\\mu$m)')
cb = plt.colorbar(mesh1, pad=0.15, label='Photon number')

mesh2 = ax[1].pcolormesh(1000*uu/maag, 1000*zz, psfy, cmap='gray', norm=LogNorm(vmin=1e-2, vmax=1e2))
ax[1].axhline(-1000*d_.cpu().numpy()[25], alpha=0.7)
ax[1].scatter([0.], 1000*z.cpu().detach().numpy()[0], label='Emitter position', c='r', marker='+')
ax[1].legend()
ax[1].set_xlim((posx-x_lim, posx+x_lim))
#ax[1].set_ylim((posz*1000-z_lim, posz*1000+z_lim))
ax[1].set_aspect('equal')
ax[1].set_xlabel('x_sample (nm)')
ax[1].set_ylabel('z (nm)')
ax[1].set_title('y polarization')
secax = ax[1].secondary_xaxis('top', functions=(lambda x : 0.001*maag*x, lambda x : 1000*x/maag))
secax.set_xlabel('x_camera ($\\mu$m)')
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')

mesh3 = ax[2].pcolormesh(1000*uu/maag, 1000*zz, psfx+psfy, cmap='gray')#, norm=LogNorm(vmin=1e-2, vmax=1e2))
ax[2].axhline(-1000*d_.cpu().numpy()[25], alpha=0.7)
ax[2].scatter([0.], 1000*z.cpu().detach().numpy()[0], label='Emitter position', c='r', marker='+')
ax[2].legend()
ax[2].set_xlim((posx-x_lim, posx+x_lim))
#ax[2].set_ylim((posz*1000-z_lim, posz*1000+z_lim))
ax[2].set_aspect('equal')
ax[2].set_xlabel('x_sample (nm)')
ax[2].set_ylabel('z (nm)')
ax[2].set_title('unpolarized')
secax = ax[2].secondary_xaxis('top', functions=(lambda x : 0.001*maag*x, lambda x : 1000*x/maag))
secax.set_xlabel('x_camera ($\\mu$m)')
cb = plt.colorbar(mesh3, pad=0.15, label='Photon number')

In [ ]:
uu, zz = np.meshgrid(u[0].cpu(), -d_.cpu().numpy())

psfx = psf[:,0,:,int(psf.shape[2]/2)].cpu().detach().numpy()
psfy = psf[:,1,:,int(psf.shape[2]/2)].cpu().detach().numpy()
psf_unpolarized = psfx+psfy

In [ ]:
x_lim = 2000#800
z_lim = 2000
plt.rcParams['figure.figsize'] = [17, 5]
fig, ax = plt.subplots(1,3)


mesh1 = ax[0].pcolormesh(1000*uu/maag, 1000*zz, psfx, cmap='gray', norm=LogNorm(vmin=1e-2, vmax=1e2))
ax[0].scatter([0.], 1000*z.cpu().detach().numpy()[0], label='Emitter position', c='r', marker='+')
ax[0].legend()
ax[0].set_xlim((posx-x_lim, posx+x_lim))
#ax[0].set_ylim((posz*1000-z_lim, posz*1000+z_lim))
ax[0].set_aspect('equal')
ax[0].set_xlabel('y_sample (nm)')
ax[0].set_ylabel('z (nm)')
ax[0].set_title('x polarization')
secax = ax[0].secondary_xaxis('top', functions=(lambda x : 0.001*maag*x, lambda x : 1000*x/maag))
secax.set_xlabel('y_camera ($\\mu$m)')
cb = plt.colorbar(mesh1, pad=0.15, label='Photon number')

mesh2 = ax[1].pcolormesh(1000*uu/maag, 1000*zz, psfy, cmap='gray', norm=LogNorm(vmin=1e-2, vmax=1e2))
ax[1].scatter([0.], 1000*z.cpu().detach().numpy()[0], label='Emitter position', c='r', marker='+')
ax[1].legend()
ax[1].set_xlim((posx-x_lim, posx+x_lim))
#ax[1].set_ylim((posz*1000-z_lim, posz*1000+z_lim))
ax[1].set_aspect('equal')
ax[1].set_xlabel('y_sample (nm)')
ax[1].set_ylabel('z (nm)')
ax[1].set_title('y polarization')
secax = ax[1].secondary_xaxis('top', functions=(lambda x : 0.001*maag*x, lambda x : 1000*x/maag))
secax.set_xlabel('y_camera ($\\mu$m)')
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')

mesh3 = ax[2].pcolormesh(1000*uu/maag, 1000*zz, psfx+psfy, cmap='gray')#, norm=LogNorm(vmin=1e-2, vmax=1e2))
ax[2].scatter([0.], 1000*z.cpu().detach().numpy()[0], label='Emitter position', c='r', marker='+')
ax[2].legend()
ax[2].set_xlim((posx-x_lim, posx+x_lim))
#ax[2].set_ylim((posz*1000-z_lim, posz*1000+z_lim))
ax[2].set_aspect('equal')
ax[2].set_xlabel('y_sample (nm)')
ax[2].set_ylabel('z (nm)')
ax[2].set_title('unpolarized')
secax = ax[2].secondary_xaxis('top', functions=(lambda x : 0.001*maag*x, lambda x : 1000*x/maag))
secax.set_xlabel('y_camera ($\\mu$m)')
cb = plt.colorbar(mesh3, pad=0.15, label='Photon number')